In [1]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

# Função 1

In [2]:
from sympy import symbols

variaveis = list(symbols('x1:8'))

print("Variáveis da função 1:")
display(variaveis)

Variáveis da função 1:


[x1, x2, x3, x4, x5, x6, x7]

In [3]:
f_x = sum([100 * (variaveis[i+1] - variaveis[i]**2)**2 + (1 - variaveis[i])**2 for i in range(len(variaveis)-1)])

print("Função 1:")
display(f_x)

Função 1:


(1 - x1)**2 + (1 - x2)**2 + (1 - x3)**2 + (1 - x4)**2 + (1 - x5)**2 + (1 - x6)**2 + 100*(-x1**2 + x2)**2 + 100*(-x2**2 + x3)**2 + 100*(-x3**2 + x4)**2 + 100*(-x4**2 + x5)**2 + 100*(-x5**2 + x6)**2 + 100*(-x6**2 + x7)**2

In [4]:
from algoritmos import calcula_gradiente

grad_f_x = calcula_gradiente(f_x, variaveis)

print("Vetor gradiente da função 1:")
display(grad_f_x)

Vetor gradiente da função 1:


[-400*x1*(-x1**2 + x2) + 2*x1 - 2,
 -200*x1**2 - 400*x2*(-x2**2 + x3) + 202*x2 - 2,
 -200*x2**2 - 400*x3*(-x3**2 + x4) + 202*x3 - 2,
 -200*x3**2 - 400*x4*(-x4**2 + x5) + 202*x4 - 2,
 -200*x4**2 - 400*x5*(-x5**2 + x6) + 202*x5 - 2,
 -200*x5**2 - 400*x6*(-x6**2 + x7) + 202*x6 - 2,
 -200*x6**2 + 200*x7]

In [5]:
from algoritmos import calcula_hessiana

hess_f_x = calcula_hessiana(f_x, variaveis)

print("Matriz Hessiana da função 1:")
display(hess_f_x)

Matriz Hessiana da função 1:


Matrix([
[1200*x1**2 - 400*x2 + 2,                   -400*x1,                         0,                         0,                         0,                         0,       0],
[                -400*x1, 1200*x2**2 - 400*x3 + 202,                   -400*x2,                         0,                         0,                         0,       0],
[                      0,                   -400*x2, 1200*x3**2 - 400*x4 + 202,                   -400*x3,                         0,                         0,       0],
[                      0,                         0,                   -400*x3, 1200*x4**2 - 400*x5 + 202,                   -400*x4,                         0,       0],
[                      0,                         0,                         0,                   -400*x4, 1200*x5**2 - 400*x6 + 202,                   -400*x5,       0],
[                      0,                         0,                         0,                         0,                   -400*x5, 12

In [6]:
# TODO: Estudo da função: pontos críticos, convexidade, existência de ponto(s) ótimo(s), além de plotar sua função, etc.

In [7]:
# TODO: Achar os melhores valores de gama e eta

In [8]:
import numpy as np
import pandas as pd

colunas = ['Algoritmo', 'Ponto inicial', '# de iteracoes', '# de cham. de armijo', 'Ponto otimo', 'Valor otimo', 'Erro de aproximação']

pontos = [
  np.array([2.9, 2.1]), 
  np.array([-3.1, -1.9]), 
  # np.array([30, 20]), 
  np.array([4, -2]), 
  np.array([-3, 1]), 
  np.array([2.5, 1.5]), 
  np.array([-3.5, 2.5]), 
  # np.array([0, 0]), 
  np.array([5, 5]), 
  np.array([3.9, -0.9])
  ]

eta = 0.02
gama = 0.08

df_base = pd.DataFrame(columns=colunas)

print("Gama:", gama)
print("Eta:", eta)

print("Pontos a serem testados:")
display(pontos)

Gama: 0.08
Eta: 0.02
Pontos a serem testados:


[array([2.9, 2.1]),
 array([-3.1, -1.9]),
 array([ 4, -2]),
 array([-3,  1]),
 array([2.5, 1.5]),
 array([-3.5,  2.5]),
 array([5, 5]),
 array([ 3.9, -0.9])]

### Gradiente

In [9]:
from algoritmos import metodo_do_gradiente
from utils import substitui_variaveis_funcao, substitui_variaveis_gradiente

df_gradiente = df_base.copy()

for ponto in pontos:
    ponto_otimo, n_iteracoes, n_iteracoes_armijo = metodo_do_gradiente(f_x, grad_f_x, variaveis, ponto, gama, eta)
    valor_otimo = substitui_variaveis_funcao(f_x, variaveis, ponto_otimo)
    erro_aproximacao = np.linalg.norm(substitui_variaveis_gradiente(grad_f_x, variaveis, ponto_otimo))
    df_gradiente = pd.concat([df_gradiente, pd.DataFrame([['gradiente', ponto, n_iteracoes, n_iteracoes_armijo, ponto_otimo, valor_otimo, erro_aproximacao]], columns=colunas)])

df_gradiente

TypeError: Cannot convert expression to float

### Newton

In [10]:
from algoritmos import metodo_de_newton

df_newton = df_base.copy()

for ponto in pontos:
    ponto_otimo, n_iteracoes, n_iteracoes_armijo = metodo_de_newton(f_x, grad_f_x, hess_f_x, variaveis, ponto, gama, eta)
    valor_otimo = substitui_variaveis_funcao(f_x, variaveis, ponto_otimo)
    erro_aproximacao = np.linalg.norm(substitui_variaveis_gradiente(grad_f_x, variaveis, ponto_otimo))
    df_newton = pd.concat([df_newton, pd.DataFrame([['newton', ponto, n_iteracoes, n_iteracoes_armijo, ponto_otimo, valor_otimo, erro_aproximacao]], columns=colunas)])

display(df_newton)

TypeError: Cannot convert expression to float

### Quase-Newton

In [11]:
from algoritmos import metodo_de_quase_newton

df_quase_newton = df_base.copy()

for metodo in ['bfgs', 'dfp']:
    for ponto in pontos:
        ponto_otimo, n_iteracoes, n_iteracoes_armijo = metodo_de_quase_newton(f_x, len(variaveis), grad_f_x, variaveis, ponto, gama, eta, metodo=metodo)
        valor_otimo = substitui_variaveis_funcao(f_x, variaveis, ponto_otimo)
        erro_aproximacao = np.linalg.norm(substitui_variaveis_gradiente(grad_f_x, variaveis, ponto_otimo))
        df_quase_newton = pd.concat([df_quase_newton, pd.DataFrame([[f'quase-newtown ({metodo})', ponto, n_iteracoes, n_iteracoes_armijo, ponto_otimo, valor_otimo, erro_aproximacao]], columns=colunas)])

display(df_quase_newton)

TypeError: Cannot convert expression to float

## Tabela geral

In [12]:
df_all = pd.concat([df_gradiente, df_newton, df_quase_newton])
df_all

,Algoritmo,Ponto inicial,# de iteracoes,# de cham. de armijo,Ponto otimo,Valor otimo,Erro de aproximação
